In [5]:
import os
import pathlib
import collections

import numpy as np
import pandas as pd

In [6]:
column_names = ["id", "raw_image_filepath", "scored_image_filepath", "sorted_image_filepath", "cpc_filepath"]
index_dict = collections.defaultdict(lambda: [None, None, None, None])

for site_name in os.listdir("C-30"):
    if (site_name == "_Scoring Excel"):
        continue

    for image_type in ["RAW", "SCORED", "SORTED"]:
        for (dirpath, _, filenames) in os.walk(os.path.join("C-30", site_name, image_type)):
            for filename in filenames:
                full_file_path = os.path.join(dirpath, filename)
                image_id = pathlib.Path(full_file_path).stem

                is_jpg = filename.lower().endswith(".jpg")
                is_cpc = filename.lower().endswith(".cpc")

                if is_jpg:
                    if image_type == "RAW":
                        index_dict[image_id][0] = full_file_path
                    elif image_type == "SCORED":
                        index_dict[image_id][1] = full_file_path
                    elif image_type == "SORTED":
                        index_dict[image_id][2] = full_file_path
                elif is_cpc:
                    index_dict[image_id][3] = full_file_path

    


In [7]:
image_ids_np = np.array(list(index_dict.keys()))
image_filepaths_np = np.array(list(index_dict.values()))

index_df = pd.DataFrame(np.c_[image_ids_np, image_filepaths_np], columns=column_names)
index_df

,id,raw_image_filepath,scored_image_filepath,sorted_image_filepath,cpc_filepath
0,PB084364,C-30\SHINE-1737(Toroso A)\RAW\OLYMPUSTG6_CBRAC...,None,None,None
1,PB084365,C-30\SHINE-1737(Toroso A)\RAW\OLYMPUSTG6_CBRAC...,None,None,None
2,PB084366,C-30\SHINE-1737(Toroso A)\RAW\OLYMPUSTG6_CBRAC...,C-30\SHINE-1737(Toroso A)\SCORED\PB084366.JPG,C-30\SHINE-1737(Toroso A)\SORTED\2\PB084366.JPG,C-30\SHINE-1737(Toroso A)\SCORED\PB084366.cpc
3,PB084367,C-30\SHINE-1737(Toroso A)\RAW\OLYMPUSTG6_CBRAC...,C-30\SHINE-1737(Toroso A)\SCORED\PB084367.JPG,C-30\SHINE-1737(Toroso A)\SORTED\2\PB084367.JPG,C-30\SHINE-1737(Toroso A)\SCORED\PB084367.cpc
4,PB084368,C-30\SHINE-1737(Toroso A)\RAW\OLYMPUSTG6_CBRAC...,C-30\SHINE-1737(Toroso A)\SCORED\PB084368.JPG,C-30\SHINE-1737(Toroso A)\SORTED\2\PB084368.JPG,C-30\SHINE-1737(Toroso A)\SCORED\PB084368.cpc
...,...,...,...,...,...
1110,PB150831,C-30\SHINE-1752(KAY REYNA)(Th)\RAW\OLYMPUSTG6_...,None,C-30\SHINE-1752(KAY REYNA)(Th)\SORTED\1\PB1508...,None
1111,PB150832,C-30\SHINE-1752(KAY REYNA)(Th)\RAW\OLYMPUSTG6_...,C-30\SHINE-1752(KAY REYNA)(Th)\SCORED\PB150832...,C-30\SHINE-1752(KAY REYNA)(Th)\SORTED\1\PB1508...,C-30\SHINE-1752(KAY REYNA)(Th)\SCORED\PB150832...
1112,PB150833,C-30\SHINE-1752(KAY REYNA)(Th)\RAW\OLYMPUSTG6_...,C-30\SHINE-1752(KAY REYNA)(Th)\SCORED\PB150833...,C-30\SHINE-1752(KAY REYNA)(Th)\SORTED\1\PB1508...,C-30\SHINE-1752(KAY REYNA)(Th)\SCORED\PB150833...
1113,PB150834,C-30\SHINE-1752(KAY REYNA)(Th)\RAW\OLYMPUSTG6_...,None,C-30\SHINE-1752(KAY REYNA)(Th)\SORTED\1\PB1508...,None


In [8]:
index_df.to_csv("c30-filename-index.csv", index=False)